In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
with open("example_sentences.txt", encoding='utf-8') as f:
            lines = f.readlines()


In [4]:
EXAMPLE_SENTENCES = lines

In [5]:
import os
with open("furniture.txt", encoding='utf-8') as f:
            lines = f.readlines()

In [6]:
furnitures = []
labels = []
furniture_id = []
id = 0 
for line in lines:
    array_of_line = line.split()
    length = len(array_of_line)
    furnitures.append(array_of_line[0])
    labels.append("B-FUR")
    furniture_id.append(id)
    if length > 1:
                for k in range(1,length):
                                        furnitures.append(array_of_line[k])
                                        labels.append("I-FUR")
                                        furniture_id.append(id)
    id = id+1
		

In [7]:
furniture_data = pd.DataFrame({
    "f_id":furniture_id,
    "word":furnitures,
    "label":labels
})
furniture_data

,f_id,word,label
0,0,Baby,B-FUR
1,0,bottle,I-FUR
2,1,Bassinet,B-FUR
3,2,Bib,B-FUR
4,3,Doll,B-FUR
...,...,...,...
174,109,table,I-FUR
175,110,Bin,B-FUR
176,111,Trash,B-FUR
177,112,Trash,B-FUR


In [8]:
sentence_id = []
labels = []
words = []

In [9]:
liste = [1,4,2,3,7]
liste.index(4)

1

In [10]:
import os
with open("additional_furniture.txt", encoding='utf-8') as f:
            lines = f.readlines()


In [11]:
additional_furnitures = []
for i in range(len(lines)):
        x = lines[i]
        x = x[:-1]
        additional_furnitures.append(x)

additional_furnitures

['storage container',
 'television',
 'sound system',
 'computer',
 'bed',
 'glasses',
 'glass',
 'billiard table',
 'blankets',
 'blanket',
 'linens',
 'linen',
 'umbrellas',
 'umbrella',
 'book',
 'books',
 'novels',
 'novel',
 'chairs',
 'mirror',
 'shelves',
 'cushions',
 'desk',
 'cabin',
 'chaise',
 'recliner',
 'seat',
 'tray',
 'tablet']

In [12]:
import random
def func_create_data(furniture_id, example_sentence_remain, id):
    if len(example_sentence_remain)<20:
        example_sentence_remain = EXAMPLE_SENTENCES.copy()
    
    lst = list(range(0,len(example_sentence_remain)))
    random_samples = random.sample(lst,20)

    sample_sentence = example_sentence_remain.copy()

    print(random_samples)
    for t in range(20):
       #print(random_samples[t])
       to_be_deleted_sentence = example_sentence_remain[random_samples[t]]
       to_be_deleted_index = sample_sentence.index(to_be_deleted_sentence)
       del sample_sentence[to_be_deleted_index]

    
    for i in range(20):
        tnk = 0
        sntnc = example_sentence_remain[random_samples[i]]
        list_of_words = sntnc.split()
        syst_bool = 0
        if any(item in sntnc for item in additional_furnitures):
            if ('storage container' in sntnc) or ('sound system' in sntnc) or ('billiard table' in sntnc):
                for t in range(len(list_of_words)):
                    k = list_of_words[t]
                    if syst_bool == 1:
                        syst_bool = 0
                        continue
                    if k == "[MASK]":
                        f_data = furniture_data[furniture_data["f_id"]==furniture_id]
                        for j in range(len(f_data)):
                            labels.append(f_data.label.iloc[j])
                            sentence_id.append(id)
                            words.append(f_data.word.iloc[j])

                    elif (k == "storage") or (k == "billiard") or (k =="sound"):
                        labels.append("B-FUR")
                        sentence_id.append(id)
                        words.append(k)
                        labels.append("I-FUR")
                        sentence_id.append(id)
                        words.append(list_of_words[t+1])
                        syst_bool = 1

                    else:
                        labels.append("O")
                        sentence_id.append(id)
                        words.append(k)
            else:
                for t in range(len(list_of_words)):
                    k = list_of_words[t]
                    if k == "[MASK]":
                        f_data = furniture_data[furniture_data["f_id"]==furniture_id]
                        for j in range(len(f_data)):
                            labels.append(f_data.label.iloc[j])
                            sentence_id.append(id)
                            words.append(f_data.word.iloc[j])

                    elif k in additional_furnitures:
                        labels.append("B-FUR")
                        sentence_id.append(id)
                        words.append(k)
                        
                    elif (("." in k)or ("?" in k)) and (t == len(list_of_words)-1):
                        tmp = k[:-1]
                        tmp2 = k[-1]

                        if tmp in additional_furnitures:
                            labels.append("B-FUR")
                            sentence_id.append(id)
                            words.append(tmp)
                        else:
                            labels.append("O")
                            sentence_id.append(id)
                            words.append(tmp)

                        labels.append("O")
                        sentence_id.append(id)
                        words.append(tmp2)

                    else:
                        labels.append("O")
                        sentence_id.append(id)
                        words.append(k)

        else:
            
            for t in range(len(list_of_words)):
                k = list_of_words[t]
                if k == "[MASK]":
                    f_data = furniture_data[furniture_data["f_id"]==furniture_id]
                    for j in range(len(f_data)):
                        labels.append(f_data.label.iloc[j])
                        sentence_id.append(id)
                        words.append(f_data.word.iloc[j])
                elif( ("." in k) or ('?' in k) )and (t == len(list_of_words)-1):
                    tmp = k[:-1]
                    tmp2 = k[-1]
                    labels.append("O")
                    labels.append("O")
                    sentence_id.append(id)
                    sentence_id.append(id)
                    words.append(tmp)
                    words.append(tmp2)


                else:
                    labels.append("O")
                    sentence_id.append(id)
                    words.append(k)
        id+=1 
    return sample_sentence,id
            
	
    

In [13]:
furniture_id_list = list(range(furniture_data.f_id.max()+1))
example_sentence_remain = EXAMPLE_SENTENCES.copy()
id = 0

In [14]:
for i in range(furniture_data.f_id.max()+1):
    example_sentence_remain,id = func_create_data(i,example_sentence_remain,id)

[34, 89, 37, 33, 45, 36, 28, 69, 11, 3, 55, 26, 97, 78, 18, 90, 41, 39, 73, 29]
[13, 63, 40, 67, 36, 58, 38, 35, 1, 57, 23, 66, 78, 61, 49, 75, 2, 39, 68, 15]
[44, 2, 58, 47, 57, 9, 12, 4, 8, 18, 49, 29, 25, 53, 15, 43, 54, 39, 38, 36]
[36, 4, 17, 13, 20, 25, 37, 18, 24, 29, 21, 2, 34, 15, 38, 23, 12, 22, 35, 3]
[7, 1, 14, 17, 10, 19, 5, 3, 18, 8, 12, 2, 13, 9, 16, 15, 11, 6, 0, 4]
[91, 70, 85, 32, 64, 59, 4, 99, 86, 0, 43, 56, 25, 98, 72, 8, 79, 48, 60, 80]
[40, 48, 58, 56, 67, 74, 78, 30, 73, 8, 68, 65, 17, 71, 28, 39, 14, 18, 42, 52]
[7, 18, 24, 52, 54, 48, 32, 13, 16, 55, 58, 28, 37, 27, 22, 29, 45, 42, 46, 31]
[36, 13, 22, 16, 17, 25, 31, 26, 2, 9, 28, 38, 33, 27, 12, 29, 39, 6, 11, 34]
[9, 2, 16, 10, 5, 4, 8, 13, 0, 17, 12, 11, 1, 3, 19, 7, 18, 6, 14, 15]
[88, 58, 6, 29, 42, 84, 99, 81, 14, 49, 89, 46, 43, 57, 25, 5, 77, 34, 28, 62]
[49, 7, 16, 12, 48, 43, 2, 66, 44, 37, 34, 75, 58, 20, 30, 10, 51, 11, 3, 74]
[45, 44, 57, 10, 29, 17, 37, 31, 49, 46, 21, 22, 34, 48, 7, 13, 8, 36, 

In [15]:
id

2260

In [16]:
data = pd.DataFrame({
    "sentence_id":sentence_id,
    "word": words,
    "label":labels
})
data

,sentence_id,word,label
0,0,The,O
1,0,Baby,B-FUR
2,0,bottle,I-FUR
3,0,in,O
4,0,the,O
...,...,...,...
33892,2259,displays,O
33893,2259,our,O
33894,2259,fine,O
33895,2259,china,O


In [17]:
data[data["sentence_id"]==0]

,sentence_id,word,label
0,0,The,O
1,0,Baby,B-FUR
2,0,bottle,I-FUR
3,0,in,O
4,0,the,O
5,0,master,O
6,0,bedroom,O
7,0,is,O
8,0,where,O
9,0,I,O


In [18]:
data.to_excel("model_data.xlsx", encoding="utf-8", index=False)